In [0]:
import cv2 as cv
import numpy as np
from keras.utils import to_categorical
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.datasets import mnist
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
gabarito = pickle.load(open( "gabarito", "rb" )) # Lê o pickle contendo as imagens e seus respectivos números, gerado no arquivo treinamento.ipynb

In [0]:
gabarito[0][1]

In [0]:
train = gabarito[0:int(len(gabarito)*0.75)] # Separa os primeiros 75% dos dados para o treinamento

In [0]:
test = gabarito[int(len(gabarito)*0.75):]# Separa os 25% finais para testas as predições

In [0]:
Xtrain, Ytrain = zip(*train) # "unzip" da lista de tuplas para duas listas, uma contendo as imagens, e a outra contendo os números

In [0]:
Xtest, Ytest = zip(*test)

In [0]:
Xlr = np.reshape(Xtrain, (len(Xtrain), 784))
Xtlr = np.reshape(Xtest, (len(Xtest), 784))

In [0]:
Ytrain = np.array(list(Ytrain), dtype='uint8')
# Ytrain

In [0]:
Ytest = np.array(list(Ytest), dtype='uint8')
# Ytest

In [0]:
y_oe = to_categorical(np.array(Ytrain), 10)
yt_oe = to_categorical(np.array(Ytest), 10)

In [0]:
Xnn = Xlr/255
Xtnn = Xtlr/255

In [0]:
Xcnn = Xnn.reshape(Xnn.shape[0], 28, 28, 1).astype('float32') # reshapes para se adequares às predições
Xtcnn = Xtnn.reshape(Xtnn.shape[0], 28, 28, 1).astype('float32')

In [0]:
model_cnn = Sequential() # criação do modelo e de suas camadas

model_cnn.add(layers.Conv2D(32, (5, 5), input_shape=(28, 28, 1), activation='relu'))
model_cnn.add(layers.MaxPooling2D(pool_size=(2, 2)))
model_cnn.add(layers.Flatten())
model_cnn.add(layers.Dense(128, activation='relu'))
model_cnn.add(layers.Dense(10, activation='softmax'))
model_cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
model_cnn.fit(Xcnn, y_oe, validation_data=(Xtcnn, yt_oe), batch_size=200, verbose=2, epochs = 3000)

In [0]:
# geração de imagens com pequenas alterações para aumentar a base de dados
imageData = ImageDataGenerator(width_shift_range=5, height_shift_range=5, rotation_range=15, zoom_range=0.15)

In [0]:
# treinamento final
model_cnn.fit_generator(imageData.flow(Xcnn, y_oe, batch_size = 200, shuffle = True), steps_per_epoch = len(Xcnn)/100, epochs = 3000)

In [0]:
model_cnn.save("modelo.h5")